In [1]:
## Levy

######### install botorch library ##########

import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import qMaxValueEntropy
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.test_functions import Levy
from botorch.optim.initializers import initialize_q_batch_nonneg
import numpy as np
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import random
from random import choices
import matplotlib.pyplot as plt
from botorch.utils.transforms import standardize, normalize, unnormalize

torch.set_default_dtype(torch.float64) # change data type to float64
torch.set_printoptions(precision=10) # change print precision

result = []

for ii in range(0,1):
    # ==================================
    # reward weight --- Ackley
    # ==================================

    num_dev = 5 # number of device
    input_dim = 8 # dimension of X
    num_sample = choices(range(input_dim*5,input_dim*5+1), k=num_dev) # random.sample(range(10, 25), num_dev)# number of initial sample
    # lr = 0.01 # learning rate

    I_matrix = np.eye(input_dim) # I matrix
    W_matrix_1 = ( np.ones((num_dev,num_dev)) ) / num_dev # np.eye(num_dev) # ( np.ones((num_dev,num_dev)) ) / num_dev # consensus matrix
    W_matrix_2 = W_matrix_1 #( np.ones((num_dev,num_dev)) ) / num_dev

    n_comm = n_comm_total = 20*input_dim
    # n_comm = 15 # np.round(n_comm_total*0.7) #20 # number of communication round
    # n_comm.astype(np.int64)
    # n_ind_comm = n_comm_total - n_comm

    bounds = torch.tensor([[-10., 10.], [-10., 10.], [-10., 10.] , [-10., 10.], [-10., 10.], [-10., 10.], [-10., 10.], [-10., 10.]]).T
    # bounds = torch.tensor([[-33., 33.], [-33., 33.], [-33., 33.], [-33., 33.], [-33., 33.] ]).T
    # bounds = torch.tensor([[-1., 1.], [-1., 1.], [-1., 1.], [-1., 1.], [-1., 1.] ]).T

    # -------------------
    # creating data
    # -------------------

    counter = 0

    from operator import itemgetter # argmax

    for i in range(0, num_dev):
        globals()['a_%s' % i]  = torch.FloatTensor(1, 1).uniform_(0.5, 1) #np.random.uniform(0.5, 1, 1)
        globals()['b_%s' % i]  = torch.empty(1).normal_(mean=0,std=1) #np.random.normal(0, 1, 1)
        globals()['c_%s' % i]  = torch.empty(1).normal_(mean=0,std=1) #np.random.normal(0, 1, 1)
        globals()['original_train_X_%s' % i] = globals()['temp_train_X_%s' % i] = globals()['train_X_%s' % i] = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(num_sample[i], input_dim) # globals() can create variable on-the-fly
        globals()['train_X_%s' % i] = eval('train_X_%s' % i) + eval('c_%s' % i)
        globals()['original_train_Y_%s' % i] = globals()['temp_train_Y_%s' % i] = globals()['train_Y_%s' % i] = Levy(dim=input_dim, negate=True)(eval('train_X_%d' % (i))).unsqueeze(-1) # eval() can directly call the variable on-the-fly
        globals()['train_Y_%s' % i] =  eval('a_%d' % (i))*eval('train_Y_%d' % (i)) +  eval('b_%d' % (i))
        globals()['current_opt_%s' % i] = eval('train_Y_%d' % (i)).max()
        globals()['optimal_Y_%s' % i] = (torch.tensor(0) *  eval('a_%d' % (i)) +  eval('b_%d' % (i)))[0][0] 

        # globals()['train_X_%s' % i] = normalize(eval('original_train_X_%d' % (i)), bounds=bounds)



    ### collaboration

    for iter in range(0, n_comm_total):

        print(iter)
        if iter == 0: loss_vector = [] # store losses (i.e., negative AF) from all devices
        # -------------------
        # initialize model
        # -------------------

        for i in range(0, num_dev):
            globals()['model_%s' % i] = SingleTaskGP(eval('train_X_%d' % (i)), eval('train_Y_%d' % (i)))
            globals()['mll_%s' % i] = ExactMarginalLogLikelihood(eval('model_%d' % (i)).likelihood, eval('model_%d' % (i)))
            fit_gpytorch_mll(eval('mll_%d' % (i)))
            globals()['candidate_set_%s' % i] = bounds[0] + (bounds[1] - bounds[0]) * (torch.rand(1000, bounds.size(1), device=bounds.device, dtype=bounds.dtype))
            globals()['qMES_%s' % i] = ExpectedImprovement(eval('model_%d' % (i)), best_f= eval('train_Y_%d' % (i)).max()) # qMaxValueEntropy(eval('model_%d' % (i)), eval('candidate_set_%d' % (i)))

            N = 1
            q = 1
            d = input_dim
            globals()['Xraw_%s' % i] = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(100 * N, q, d)
            globals()['Yraw_%s' % i] = eval('qMES_%d' % (i))(eval('Xraw_%d' % (i)))  # acquisition function

            if iter == 0:
                globals()['X_%s' % i], globals()['loss_%s' % i] = optimize_acqf(
                    eval('qMES_%d' % (i)), bounds=bounds, q=1, num_restarts=10, raw_samples=512,
                )
                loss_vector.append(eval('loss_%d' % (i)))
            # if iter >=1 and eval('loss_change_%d' % (i)) == 1:
            #     globals()['X_%s' % i] = initialize_q_batch_nonneg(eval('Xraw_%d' % (i)), eval('Yraw_%d' % (i)), N)
            #     counter += 1
            # globals()['X_%s' % i] = initialize_q_batch_nonneg(eval('Xraw_%d' % (i)), eval('Yraw_%d' % (i)), N)
            # eval('X_%d' % (i)).requires_grad_(True)  # require grad
            # globals()['optimizer_%s' % i] = torch.optim.SGD([eval('X_%d' % (i))], lr=lr)  # optimizer, can use sgd or adam
            globals()['X_traj_%s' % i] = []  # store sgd trajectory

        index, element = max(enumerate(loss_vector), key=itemgetter(1))
        loss_vector = []  # store losses (i.e., negative AF) from all devices
        # -------------------
        # running sgd
        ## we assume all devices are connected and consensus step only happened once
        ## we run multiple steps of local SGD
        # -------------------

        X_total = []
        for i in range(0, num_dev):
            X_temp = eval('X_%d' % (i)).detach().numpy()
            X_total = np.append(X_total, X_temp)

        if W_matrix_1[index, index] > ((num_dev-1)**2)/(n_comm*num_dev):
            W_matrix_2 = W_matrix_1 - 1/(n_comm*num_dev)
            W_matrix_2[:, index] = W_matrix_1[:, index] + (num_dev-1)/(n_comm*num_dev)
            W_matrix_2[index, :] = W_matrix_1[index, :] + (num_dev - 1) / (n_comm * num_dev)
            W_matrix_2[index, index] = W_matrix_1[index, index] - ((num_dev-1)**2)/(n_comm*num_dev)
        else:
            W_matrix_2 = np.zeros((num_dev,num_dev)) + (num_dev-2)/(num_dev-1)**2
            W_matrix_2[:, index] = 1/(num_dev-1)
            W_matrix_2[index, :] = 1/(num_dev-1)
            W_matrix_2[index, index] = 0
            print("change")
        W_matrix_2 = W_matrix_1 # delete 
        consensus = np.kron(W_matrix_2, I_matrix)  # W kron I
        updated_X_total = np.matmul(consensus, X_total)  # consensus step
        weight_increase = (num_dev-1)/(num_dev*n_comm)
        W_matrix_increase = np.eye(num_dev)*weight_increase
        weight_decrease = 1/(num_dev*n_comm)
        W_matrix_decrease = np.ones((num_dev, num_dev)) * weight_decrease
        np.fill_diagonal(W_matrix_decrease, 0)
        W_matrix_1 = W_matrix_1 + W_matrix_increase - W_matrix_decrease
        # consensus = np.kron(W_matrix_2, I_matrix)  # W kron I

        for i in range(0, num_dev):

            new_X = torch.from_numpy(updated_X_total[(i * input_dim): ((i + 1) * input_dim)])
            new_X = new_X.reshape(1, input_dim)
            temp_X = new_X
            temp_X = temp_X + eval('c_%d' % (i))
            new_Y = eval('a_%d' % (i))*Levy(dim=input_dim, negate=True)(temp_X).unsqueeze(-1) + eval('b_%d' % (i))

            # globals()['train_X_%s' % i] = torch.cat((eval('temp_train_X_%d' % (i)), new_X), dim=0) ##
            # globals()['train_Y_%s' % i] = torch.cat((eval('temp_train_Y_%d' % (i)), new_Y), dim=0) ##
            # globals()['temp_train_X_%s' % i] = eval('train_X_%d' % (i))
            # globals()['temp_train_Y_%s' % i] = eval('train_Y_%d' % (i))
            # globals()['train_X_%s' % i] = normalize(eval('train_X_%d' % (i)), bounds=bounds)
            globals()['train_X_%s' % i] = torch.cat((eval('train_X_%d' % (i)), new_X), dim=0)
            globals()['train_Y_%s' % i] = torch.cat((eval('train_Y_%d' % (i)), new_Y), dim=0)

            globals()['model_%s' % i] = SingleTaskGP(eval('train_X_%d' % (i)), eval('train_Y_%d' % (i)))
            globals()['mll_%s' % i] = ExactMarginalLogLikelihood(eval('model_%d' % (i)).likelihood, eval('model_%d' % (i)))
            fit_gpytorch_mll(eval('mll_%d' % (i)))
            globals()['candidate_set_%s' % i] = bounds[0] + (bounds[1] - bounds[0]) * (
                torch.rand(1000, bounds.size(1), device=bounds.device, dtype=bounds.dtype))
            globals()['qMES_%s' % i] = ExpectedImprovement(eval('model_%d' % (i)), best_f= eval('train_Y_%d' % (i)).max()) # qMaxValueEntropy(eval('model_%d' % (i)), eval('candidate_set_%d' % (i)))

            N = 1
            q = 1
            d = input_dim
            globals()['Xraw_%s' % i] = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(100 * N, q, d)
            globals()['Yraw_%s' % i] = eval('qMES_%d' % (i))(eval('Xraw_%d' % (i)))  # acquisition function
            globals()['X_%s' % i], globals()['loss_%s' % i] = optimize_acqf(
                eval('qMES_%d' % (i)), bounds=bounds, q=1, num_restarts=10, raw_samples=512,
            )
            loss_vector.append(-eval('loss_%d' % (i)).detach().numpy())
            

    

    average_gap = 0
    for i in range(0, num_dev):
        
        globals()['end_opt_%s' % i] = eval('train_Y_%d' % (i)).max()
        Gap = (eval('current_opt_%d' % (i))-eval('end_opt_%d' % (i)))/(eval('current_opt_%d' % (i))-eval('optimal_Y_%d' % (i)))
        result.append(Gap)
        average_gap += Gap
        print(  Gap  )

    # print(average_gap/num_dev)
    # print(result)
    # result

c:\Users\maxyxb\AppData\Local\Continuum\anaconda3\envs\consensus_BO_39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
tensor(0.9775367996)
tensor(0.9629900014)
tensor(0.9451589468)
tensor(0.9815509197)
tensor(0.9211800458)
